In [1]:
import numpy as np
import cPickle
import nltk
from nltk.corpus import stopwords
from collections import Counter
from sklearn.metrics.pairwise import pairwise_distances
import itertools
import string
from nltk.tokenize import sent_tokenize
from nltk.stem.porter import *
import os
#import pyemd
#loading word embeddings
#wv = KeyedVectors.load_word2vec_format("../w2v/w2vemb.bin", binary=True)

# Visualize functions

In [2]:
def write_file(n_clusters_, labels, corpus, mapping_dict, test_corpus):
    for indice_cluster in range(n_clusters_):
        # print indice_cluster
        idx_list = np.where(labels == indice_cluster)[0]
        f1 = open('cluster_%s/%dth_cluster.txt' % (question_class, indice_cluster), 'w+')
        for idx in idx_list:
            f1.write('%s %s\n' % (corpus[mapping_dict[idx] - 1], test_corpus[idx]))
    f1.close()


def write_ne(nes, question_class, mapping_dict):
    f1 = open('ne_%s.txt' % question_class, 'w+')
    for idx, doc in enumerate(nes):
        f1.write('%s %s\n' % (mapping_dict[idx], ' '.join(doc)))
    f1.close()    

def wmd_compute(x1, x2):
    return wv.wmdistance(x1, x2)

def analyze_cluster(n_clusters_, labels, corpus, mapping_dict, test_corpus, filename):
    num = 0
    other_content = []
    for indice_cluster in range(n_clusters_):
        idx_list = np.where(labels == indice_cluster)[0]
        if len(idx_list) > 4:
            f1 = open('%s_%s/%dth_cluster.txt' % (filename, question_class, indice_cluster), 'w+')
            for idx in idx_list:
                f1.write('%s %s\n' % (corpus[mapping_dict[idx] - 1], test_corpus[idx]))
            f1.close()
        if len(idx_list) < 4:
            num = num + len(idx_list)
            for idx in idx_list:
                other_content.append(corpus[mapping_dict[idx] - 1])
    f1 = open('%s_%s/other_cluster.txt' % (filename, question_class), 'w+')
    for doc in other_content:
        f1.write('%s\n' % doc)
    f1.close()
    print num

# Reading Raw text

In [3]:
question_class = 'q3'
fname = '..//data//%s.txt' % question_class
with open(fname) as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
content = [x.strip() for x in content]
print 'length of content', len(content)
f.close()
print content

length of content 309
['earlier , i had given my car to servicing they told me , \\? we will give complete details within 1 or 2 day or till monday \\? . after that they committed me that we will complete the servicing till friday handover car to you but it was not completed . they completed servicing on next friday . previously they checked problem and they were aware about my car door issue . after that they had not given me proper time frame . they should know how much time it will take to deliver the car back . they should say , \\? sir it will take 15 days , so please adjust to do your work . they finally gave our car on saturday .', 'they should explain everything', 'they should give proper explanation .', "they are not providing the correct information . they do n't have a capability to estimate what problem is there in the system .", 'after resolving the customer query customer should be satisfied after trial on car .', 'when customer come in showroom that time staff should qui

In [4]:
# # split comment with multi-sentence into multi-comments
# content_new = []
# for comment in content:
#     sents = sent_tokenize(comment)
#     if len(sents) > 1:
#         for i in range(len(sents)):
#             content_new.append(sents[i])
#     else:
#         content_new.append(comment)
# print 'length of new content', len(content_new)
# # print content_new
    

In [5]:
pos_new = nltk.pos_tag(nltk.word_tokenize("it is 10 km far away from city it should be nearby city within 2 3 km . "))
print pos_new

[('it', 'PRP'), ('is', 'VBZ'), ('10', 'CD'), ('km', 'NN'), ('far', 'RB'), ('away', 'RB'), ('from', 'IN'), ('city', 'NN'), ('it', 'PRP'), ('should', 'MD'), ('be', 'VB'), ('nearby', 'JJ'), ('city', 'NN'), ('within', 'IN'), ('2', 'CD'), ('3', 'CD'), ('km', 'NN'), ('.', '.')]


In [6]:
stemmer = PorterStemmer()
print stemmer.stem("explanations")

explan


In [7]:

def rule_q1(sen, ne):
    clean_ne = list(set(ne))
#     remove_words = ["improv", "custom", "servic", "peopl","person","facil","avail","good",\
#                     "center","centr","car", "dealership", "vehicl", "toyota", "problem","work", "much",\
#                    "thing", "possibl","need"]   #stemmed
    remove_words = ['custom', 'car', 'vehicl','servic', 'explain','explan']
    clean_ne = [word for word in clean_ne if word not in remove_words]
    
    save_words = []  #stemmed
    clean_ne = clean_ne + [stemmer.stem(word) for word in sen.split() if stemmer.stem(word) in save_words]
    
    # rules to merge keywords:
    
        
    clean_ne = list(set(clean_ne))
    return clean_ne

def clean_corpus(x1):
    x_new = []
    for x in x1:
        if 'no improvement' in x:
            continue
        else:
            x_new.append(x)
    return x_new

def process_corpus(x1, pos_tags, general_stop):
    # split three categories: 1 no improvemnt 2 with noun 3 others
    doc_noimprove = []
    doc_nn = []
    nn_extracted = []
    doc_other = []
    for x in x1:
        if 'no improvement' in x:
            doc_noimprove.append(x)
        else:
                nn_list = []
                sen = x
                pos_new = nltk.pos_tag(nltk.word_tokenize(sen))
                for token in pos_new:
                    if token[1] in pos_tags and not token[0] in general_stop:
                        nn_list.append(token[0])
                #stemming
                for counter,word in enumerate(nn_list):
                    nn_list[counter] = stemmer.stem(word)
                # apply rule
                nn_list = rule_q1(sen, nn_list)
                    
                if nn_list != []:
                    nn_extracted.append(nn_list)
                    doc_nn.append(sen)
                else:
                    doc_other.append(sen)
    return doc_noimprove, [doc_nn, nn_extracted], doc_other
        
nn_corpus = []

# English stop words lists
stop_words = stopwords.words('english')
punctuation_list = [unicode(i) for i in string.punctuation]
for punctuation in punctuation_list:
    stop_words.append(punctuation)

pos_tags = ['NN', 'NNS','JJ','JJR','JJS', 'VB','VBD','VBG','VBN','VBP','VBZ']

doc1, doc2, doc3 = process_corpus(content, pos_tags, stop_words)

doc_nn, nn_extracted = doc2[0], doc2[1]
print 'Comment with nn', len(doc_nn)
print 'no comments', len(doc1)
print 'Comment without nn', len(doc3)
print nn_extracted

Comment with nn 300
no comments 8
Comment without nn 1
[[u'pleas', 'give', 'frame', 'say', u'complet', u'deliv', 'proper', u'handov', 'saturday', u'awar', 'given', u'check', u'detail', 'next', 'till', 'much', 'take', 'told', 'door', 'monday', 'sir', 'friday', 'know', 'day', 'work', u'issu', 'adjust', 'time', u'commit', 'problem', 'gave'], [u'everyth'], ['proper', 'give'], [u'inform', u'provid', 'system', u'capabl', u'estim', 'problem', 'correct'], [u'satisfi', u'queri', u'resolv', 'trial'], ['respond', 'give', u'inform', 'time', 'proper', 'showroom', 'come', 'staff'], ['lie', 'say', 'fact'], [u'origin', u'awar', 'use', 'center', u'materi', 'visit', u'manufactur', u'solv', u'part', 'place', 'need', 'problem'], [u'fals', 'much', 'time', 'need', u'commit', u'updat'], [u'entranc', 'made', 'attend', 'give', u'receiv', u'queri', u'guidanc', u'avail', u'behav', 'proper', u'staff'], [u'satisfi', 'good', u'qualiti', 'advisor'], [u'fulfil', u'make', 'advisor', u'commit', 'tell', 'first'], ['give

In [8]:
from collections import Counter
def df_count(x1):
    text = []
    for ab in x1:
        text = text + ab
    df = Counter(text)
    return df
df = df_count(nn_extracted)
print df

Counter({'give': 90, 'staff': 78, 'problem': 73, u'inform': 70, 'time': 63, 'work': 60, 'proper': 60, 'need': 56, 'advisor': 45, 'tell': 40, u'part': 38, 'take': 37, 'told': 35, 'ask': 34, 'good': 33, 'done': 33, 'center': 33, 'know': 33, 'thing': 32, u'detail': 32, u'improv': 31, 'dealership': 31, u'check': 31, 'given': 30, u'chang': 27, u'issu': 25, u'provid': 25, 'come': 24, 'said': 20, 'person': 20, u'charg': 19, 'get': 19, u'knowledg': 19, u'want': 18, u'advis': 18, 'understand': 17, 'go': 16, u'satisfi': 16, u'advic': 16, 'call': 15, 'use': 14, u'complet': 14, 'answer': 13, 'talk': 13, u'anyth': 13, 'correct': 13, 'say': 13, u'everyth': 13, u'guid': 13, 'toyota': 13, 'much': 13, u'appoint': 13, 'visit': 12, u'train': 12, 'clear': 12, 'drive': 12, u'requir': 12, u'avail': 12, 'new': 11, 'wrong': 11, 'first': 11, 'suggest': 11, 'repair': 11, u'regard': 11, u'peopl': 10, 'pay': 10, u'respons': 10, u'solv': 10, u'queri': 9, 'taken': 9, 'reason': 9, 'place': 9, 'way': 9, 'face': 9, u'

# rule-based clustering

In [9]:
stemmer = PorterStemmer()
print stemmer.stem("advisor")

advisor


In [10]:
def write_file(corpus, idx_list, word):
    if not os.path.isdir("cluster_rule/%s" %word):
            os.mkdir("cluster_rule/%s" %word)
    f1 = open('cluster_rule/%s/%s_comment.txt' % (word, word), 'w+')
    for idx in idx_list:
        f1.write('%s\n' %corpus[idx])
    f1.close()

if (not os.path.isdir("cluster_rule")):
    os.mkdir("cluster_rule")
name_list = {}
clustered_index = []

cluster_rule_list = ['inform', 'advis']
for word in cluster_rule_list:
    idx_set = []
    for idx, doc in enumerate(nn_extracted):
        if word in doc:
            idx_set.append(idx)
        write_file(doc_nn, idx_set, word)
        name_list[word] = idx_set
        clustered_index = clustered_index + idx_set

clustered_index = set(clustered_index)
unclustered_index = list(set(range(len(nn_extracted))) - clustered_index)
print "No. of clustered sentences:", len(clustered_index)
print "No. of unclustered sentences:", len(unclustered_index)


No. of clustered sentences: 85
No. of unclustered sentences: 215


# clustering using LSI

In [11]:
from gensim import corpora
from gensim.models import TfidfModel
from gensim.models import LsiModel
from gensim.similarities import MatrixSimilarity
from sklearn.metrics.pairwise import cosine_similarity

nn_extracted = [nn_extracted[i] for i in unclustered_index]
doc_nn = [doc_nn[i] for i in unclustered_index]

dictionary = corpora.Dictionary(nn_extracted)
corpus_gensim = [dictionary.doc2bow(doc) for doc in nn_extracted]
tfidf = TfidfModel(corpus_gensim)
corpus_tfidf = tfidf[corpus_gensim]
lsi = LsiModel(corpus_tfidf, id2word=dictionary, num_topics=100)
lsi_proj = lsi[corpus_tfidf]
lsi_index = MatrixSimilarity(lsi[corpus_tfidf])
similarity_matrix = np.array([lsi_index[lsi[corpus_tfidf[i]]]
                                for i in range(len(nn_extracted))])

def f(x):
    return (x+1)/2
f = np.vectorize(f, otypes=[np.float])
similarity_matrix = f(similarity_matrix)     # convert similarity_matrix from -1~1 to 0~1
print similarity_matrix

Using TensorFlow backend.


[[ 1.          0.49680785  0.56632949 ...,  0.49448629  0.52115209
   0.52640982]
 [ 0.49680786  1.          0.50078453 ...,  0.49867359  0.49659365
   0.49783689]
 [ 0.56632949  0.50078453  1.00000006 ...,  0.49817372  0.54714853
   0.55752085]
 ..., 
 [ 0.49448628  0.4986736   0.49817373 ...,  1.          0.50762646
   0.5037051 ]
 [ 0.52115209  0.49659364  0.54714853 ...,  0.50762646  1.          0.51232713]
 [ 0.52640982  0.49783689  0.55752084 ...,  0.5037051   0.51232713
   0.99999997]]


In [12]:
from sklearn.cluster import SpectralClustering
import os

n_clusters_ = 10
sc = SpectralClustering(n_clusters=n_clusters_, affinity='precomputed').fit(similarity_matrix)
labels = sc.labels_
print('Estimated number of clusters: %d' % n_clusters_)
print len(labels)

def write_file(n_clusters_, labels, doc_nn, nn_extracted):
    if (not os.path.isdir("cluster_%s" %question_class)):
        os.mkdir("cluster_%s" %question_class)
    for indice_cluster in range(n_clusters_):
        # print indice_cluster
        idx_list = np.where(labels == indice_cluster)[0]
        f1 = open('cluster_%s/%dth_cluster.txt' % (question_class, indice_cluster), 'w+')
        for idx in idx_list:
            f1.write('%s %s\n' % (doc_nn[idx], nn_extracted[idx]))
    f1.close()
    print("clusters written to folder: cluster_%s" %question_class)
    
write_file(n_clusters_, labels, doc_nn, nn_extracted)

Estimated number of clusters: 10
215
clusters written to folder: cluster_q3
